# Final Project
## Part 1: Identifying popular news sources
Author: Malika Parkhomchuk

### Tasks:
1. Gather recent tweets according to query words
2. Gather expanded urls from tweets
4. Create a data frame of all the gathered links and their source domains
5. Get the top 30 sources by the frequency of occurrence

### Task 1: Data Collection (Gather Tweets)
Setting up Twitter API Client

In [28]:
import tweepy
import re
import requests

In [29]:
my_bearer_token='AAAAAAAAAAAAAAAAAAAAAC2JWQEAAAAABPjxkEAyiEcohSqSNsXtIVpQqpM%3DHbSEkLilR2VJyGb1bmwhA36RWPqAQtE2bm0giycJoZe1uam9gF'
my_consumer_key='WNXvk96KSIQczZ3Anob4pz5xH'
my_consumer_secret='iCjMFDcFyDSlqEnwqmlRhaeE4NTziqbAJ8sAZJl3U7DuQSfa7F'
my_access_token='1461340832955609092-vgofZmqR7PMCS9lTXqpDqvip32ZKX8'
my_access_token_secret='vbTLysReQcpdxjegwvnA7xjwbuUbmVyjH3QhgRbhJGPH3'

In [30]:
client = tweepy.Client(bearer_token=my_bearer_token,
                       consumer_key=my_consumer_key, 
                       consumer_secret=my_consumer_secret, 
                       access_token=my_access_token, 
                       access_token_secret=my_access_token_secret
                      )

These are the guery words in russian and kazakh languages that I came up with by using Google Trends
for searching relevant tweets

In [31]:
queries = ['назарбаев', 'токаев', 'казахстан','аблязов', 'демократия', 'билік','саясаткер',
           'Жаңаөзен', 'Әблязов', 'коррупция', 'чиновник', 'Китай', 'коронавирус',
          'вакцина', 'вакцинация', 'минздрав', 'суицид', 'Нур Отан', 'партия', 'акжол', 'орыс', 'русофобия', 'ұлтшылдық',
           'Жанаозен', 'Қазақстан', 'ЕНПФ', 'парақорлық', 'белсенділер', 'активист', 'қудалау', 'сайлау', 'наразылық',
           'қарсылық', 'Қытай', 'Ресей', 'зейнетақы', 'қаржы', 'жарылыс', 'ынтымақтастық']

Merging query words together in order to find all of the relevant tweets

In [32]:
query_string = ''

for i in range(len(queries) - 1):
    query_string +=  queries[i] + " OR "
    
query_string += queries[-1] + ' -is:retweet has:links'

In [33]:
query_string

'назарбаев OR токаев OR казахстан OR аблязов OR демократия OR билік OR саясаткер OR Жаңаөзен OR Әблязов OR коррупция OR чиновник OR Китай OR коронавирус OR вакцина OR вакцинация OR минздрав OR суицид OR Нур Отан OR партия OR акжол OR орыс OR русофобия OR ұлтшылдық OR Жанаозен OR Қазақстан OR ЕНПФ OR парақорлық OR белсенділер OR активист OR қудалау OR сайлау OR наразылық OR қарсылық OR Қытай OR Ресей OR зейнетақы OR қаржы OR жарылыс OR ынтымақтастық -is:retweet has:links'

Getting the first page of tweets according to our query string

In [34]:
page1 = client.search_recent_tweets( 
                              query=query_string,
                              tweet_fields=['context_annotations', 
                                            'created_at',
                                            'entities',
                                            'public_metrics',
                                           ], 
                              expansions='author_id',
                              user_fields=['username', 'description', 'name', 
                                           'location', 'created_at', 
                                           'public_metrics', 'verified'],
                              max_results=100)

In [35]:
def prepareData(page):
    """Convert the objects into JSON-serializable stuff that we can easily store.
    To understand the function, we need to know the structure of the parameter page.
    """
    alldata = {'tweets': [], 'users': []}
    
    for tweet in page.data:
        tweetDct = {fld: tweet[fld] for fld in tweet}
        tweetDct['created_at'] = str(tweetDct['created_at']) # convert datatime object
        alldata['tweets'].append(tweetDct)
        
    for user in page.includes.get('users'):
        userDct = {key: user[key] for key in user}
        userDct['created_at'] = str(userDct['created_at'])
        alldata['users'].append(userDct)
            
    return alldata 

In [36]:
import json

In [37]:
data = prepareData(page1)

with open("results_0.json", 'w') as outf:
    json.dump([data], outf)

Gather tweets from further pages

In [39]:
import time
import datetime

# Helper variables
prev_page = page1
counter = 1
allPages = []

while 'next_token' in prev_page.meta:
    nextToken = prev_page.meta['next_token']
    try: 
        page = client.search_recent_tweets( 
                                  query=query_string,
                                  tweet_fields=['context_annotations', 
                                                'created_at',
                                                'entities',
                                                'public_metrics',
                                               ], 
                                  expansions='author_id',
                                  user_fields=['username', 'description', 'name', 
                                               'location', 'created_at', 
                                               'public_metrics', 'verified'],
                                  next_token=nextToken,
                                  max_results=100)
        
        # Setup things for next iteration
        prev_page = page
        # Prepare data for storage
        data = prepareData(page)
        allPages.append(data)
        # Save into a file every 1000 tweets
        if len(allPages) >= 450:
            with open(f"results_{counter}.json", 'w') as outf:
                json.dump(allPages, outf)
            allPages = []
            counter += 1
        
    except:
        print(str(datetime.datetime.now()),
                  "I hit the rate limit, going to sleep for 15 minutes.")
        time.sleep(15 * 60) # sleep for 15 minutes

# Store the remaining pages, because the rate limit was not hit, so we have unstored data        
if len(allPages) > 0:
    with open(f"results_{counter}.json", 'w') as outf:
        json.dump(allPages, outf)

In [25]:
from collections import defaultdict

In [44]:
with open("results_0.json", 'r') as outf:
    res0 = json.load(outf)

In [45]:
with open("results_1.json", 'r') as outf:
    res1 = json.load(outf)

In [48]:
len(res1)

379

In [62]:
res1.extend(res0)

In [63]:
len(res1)

380

Access dictionary values in each tweet to get expanded urls and save them in a dictionary

In [72]:
expandedUrls = defaultdict(list)

for batch in range(len(res1)):
    for tweet in res1[batch]['tweets']:
        if 'entities' in tweet.keys() and 'urls' in tweet['entities'].keys():
            for l in tweet['entities']['urls']:
                expandedUrls[tweet['id']].append(l['expanded_url'])

In [73]:
len(expandedUrls)

11720

In [74]:
out_file = open("expandedUrls-modified.json", "w")
json.dump(expandedUrls, out_file, indent = 6)
out_file.close()

A function to get the `redirectedlinks` dictionary from a JSON file

In [6]:
def getRedirectedLinksJson():
    """
    Loads the json file and
    returns its content as a variable
    """
    with open('expandedUrls-modified.json', 'r') as outputF:
        result = json.load(outputF)
    return result

In [7]:
expandedUrls = getRedirectedLinksJson()

### Task 2: Creating a Data Frame

I created a data frame from the `expandedUrls` dictionary by looping through each key in the dictionary and creating a data frame for each, and then concating all dataframes together.

In [75]:
import pandas as pd

In [76]:
alllinksdf = pd.DataFrame(columns=['tweetId', 'url'])

for key in expandedUrls:
    inter = pd.DataFrame({'tweetId': [key]*len(expandedUrls[key]), 'url': expandedUrls[key]})
    alllinksdf = pd.concat([alllinksdf,inter], ignore_index=True)
    
alllinksdf

,tweetId,url
0,1477581724481736708,http://amp.gs/jtwhq
1,1477581700070739968,https://racurs.ua/n165220-veroyatnost-lokdauna...
2,1477581519552143360,https://rus.azattyk.org/a/31635247.html
3,1477581458965475333,https://kaz.tengrinews.kz/events/shko-da-jol-a...
4,1477581279176577030,https://twitter.com/Also__O/status/14775812791...
...,...,...
15366,1477582771832954880,https://twitter.com/tominby/status/14775827718...
15367,1477582698541633536,https://ytro.news/news/coronavirus/2022/01/02/...
15368,1477582434296283139,https://twitter.com/galina12862063/status/1477...
15369,1477582434296283139,https://twitter.com/galina12862063/status/1477...


Then I added a new column `source` by extracting the domain of the link

In [78]:
alllinksdf['source'] = alllinksdf['url'].apply(lambda x: x.split("www.")[-1].split("//")[-1].split('/')[0])
alllinksdf

,tweetId,url,source
0,1477581724481736708,http://amp.gs/jtwhq,amp.gs
1,1477581700070739968,https://racurs.ua/n165220-veroyatnost-lokdauna...,racurs.ua
2,1477581519552143360,https://rus.azattyk.org/a/31635247.html,rus.azattyk.org
3,1477581458965475333,https://kaz.tengrinews.kz/events/shko-da-jol-a...,kaz.tengrinews.kz
4,1477581279176577030,https://twitter.com/Also__O/status/14775812791...,twitter.com
...,...,...,...
15366,1477582771832954880,https://twitter.com/tominby/status/14775827718...,twitter.com
15367,1477582698541633536,https://ytro.news/news/coronavirus/2022/01/02/...,ytro.news
15368,1477582434296283139,https://twitter.com/galina12862063/status/1477...,twitter.com
15369,1477582434296283139,https://twitter.com/galina12862063/status/1477...,twitter.com


Filtering unwanted links that are not news sources

In [112]:
only_news = alllinksdf[(alllinksdf['source'] != 'twitter.com') & 
                       (alllinksdf['source'] != 'youtu.be') &
                       (alllinksdf['source'] != 't.me') &
                       (alllinksdf['source'] != 'vk.com') &
                       (alllinksdf['source'] != 'tinyurl.com') &
                       (alllinksdf['source'] != 'instagram.com') &
                       (alllinksdf['source'] != 'youtube.com') &
                       (alllinksdf['source'] != 'facebook.com') &
                       (alllinksdf['source'] != 'alpemforum10.wordpress.com')
                      ]

In [113]:
only_news['ending'] = only_news['source'].apply(lambda x: x.split(".")[-1])

<ipython-input-113-04f2fdb616e0>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Filtering urls that end with "ru" and other irrelevant endings

In [121]:
only_news = only_news[(only_news.ending != 'ru') &
                      (only_news.ending != 'ly') &
                      (only_news.ending != 'tt') &
                      (only_news.ending != 'gs') &
                      (only_news.ending != 'ua') &
                      (only_news.ending != 'gd') &
                      (only_news.ending != 'by') &
                      (only_news.ending != 'website') &
                      (only_news.ending != 'it') &
                      (only_news.ending != 'rs') &
                      (only_news.ending != 'pw') &
                      (only_news.ending != 'il') &
                      (only_news.ending != 'tr') &
                      (only_news.ending != 'tv')
                     ]

In [122]:
only_news

,tweetId,url,source,ending
2,1477581519552143360,https://rus.azattyk.org/a/31635247.html,rus.azattyk.org,org
3,1477581458965475333,https://kaz.tengrinews.kz/events/shko-da-jol-a...,kaz.tengrinews.kz,kz
9,1477580880797474824,https://rus.azattyk.org/a/31636130.html,rus.azattyk.org,org
12,1477580523232968704,https://mignews.com/news/disasters/u-deputata-...,mignews.com,com
15,1477580433361567745,https://www.trtrussian.com/mnenie/soyuzniki-ka...,trtrussian.com,com
...,...,...,...,...
15310,1475057043161681920,https://politinform.net/stolica/koronavirus-v-...,politinform.net,net
15317,1475055146514227200,https://voshod.org/videochanel/broadcasts/%d0%...,voshod.org,org
15320,1475055035243372546,https://www.nur.kz/incident/crime/1948348-hisc...,nur.kz,kz
15331,1477588763136217088,https://rus.delfi.ee/statja/95544439/za-sutki-...,rus.delfi.ee,ee


### Task 3: Top 30 sources

I used the `Counter` to get the frequecy of ocurrence of each source. Then by using the counter dictionary I created a dataframe, sorted it and shown 30 top results.

In [98]:
from collections import Counter

In [123]:
counts = Counter(only_news['source'])
top_sources_df = pd.DataFrame({'news_sources': counts.keys(), 'frequency': counts.values()})
top_sources_df.sort_values(by='frequency', ascending = False, inplace = True, ignore_index = True)
top_sources_df[:30]

,news_sources,frequency
0,ktk.kz,188
1,inbusiness.kz,88
2,inform.kz,74
3,bolshoyforum.com,66
4,defence-line.org,57
5,tengrinews.kz,55
6,bbc.in,54
7,24.kz,52
8,kaz.tengrinews.kz,45
9,gordonua.com,42


In [81]:
import plotly.express as px

In [124]:
fig = px.bar(top_sources_df[:30], x='news_sources', y='frequency')
fig.show()

Using markdown to add the screenshot of the bar plot, because it doesn'y shoe up in HTML file

![top 30 news sources bar chart](bar-plot.png "image Title")

### Conclusion for Part 1 of the Final Project:

Most of the links were retweets of other tweets, so that was not very helpful for my investigation of news sources.

However, there are still alot of interesting sources to look at. 

For instance, `egov.press` is Kazakhstan's governmental source for getting information on documentations, pension, social security and etc.

`tengrinews.kz`, `inbusiness.kz`, `kaz.orda.kz`, `egemen.kz`, `akorda.kz`, `inform.kz`, `ktk.kz`, `baigenews.kz` are big news portals that mostly work of governmental orders.

`azattyq.kz`, `vlast.kz` are knows and popular independent sources of information.

Also, people from Kazakstan use `Youtube`, `Instagram` and `TikTok` to look out for the news.